In [ ]:
import numpy as np
import pandas as pd

In [ ]:
prefix = '/data/sx/erc1155-graph/wholegraph'
prefix = '/data/sx/NFTGraph'

In [ ]:
df = pd.read_csv(prefix+'/allInterEdges.csv')

In [ ]:
df.columns

In [ ]:
df

In [ ]:
df['OutCnt'] = 0
df['InCnt'] = 0
df['OutAmount'] = 0
df['InAmount'] = 0
df['OutValue'] = 0
df['InValue'] = 0
df['OutTransFee'] = 0
df['InTransFee'] = 0

In [ ]:
df['OutCnt'] = df.groupby(['from'])['OutCnt'].transform("count")

In [ ]:
df['InCnt'] = df.groupby(['to'])['InCnt'].transform("count")

In [ ]:
df['OutAmount'] = df.groupby(['from'])['transferedAmount'].transform(sum)
df['InAmount'] = df.groupby(['to'])['transferedAmount'].transform(sum)

In [ ]:
df['OutValue'] = df.groupby(['from'])['value'].transform(sum)
df['InValue'] = df.groupby(['to'])['value'].transform(sum)

In [ ]:
df['OutTransFee'] = df.groupby(['from'])['transactionFee'].transform(sum)
df['InTransFee'] = df.groupby(['to'])['transactionFee'].transform(sum)

In [ ]:
df

In [ ]:
dffrom = df[['from','OutCnt','OutAmount','OutValue','OutTransFee']]

In [ ]:
dfto = df[['to','InCnt','InAmount','InValue','InTransFee']]

In [ ]:
dffrom = dffrom.drop_duplicates(subset=['from'],ignore_index=True,keep='first')

In [ ]:
dffrom

In [ ]:
dfto = dfto.drop_duplicates(subset=['to'],ignore_index=True,keep='first')

In [ ]:
dfto

In [ ]:
dfmerge = pd.merge(dffrom,dfto,how='outer',left_on='from',right_on='to')

In [ ]:
dfmerge

In [ ]:
def getaddr(line):
    return line['from'] if str(line['to']) == 'nan' else line['to']

In [ ]:
t = dfmerge.apply(getaddr,axis=1)

In [ ]:
dfmerge['addr'] = t

In [ ]:
dfmerge

In [ ]:
dfout = dfmerge[['addr','OutCnt','OutAmount','OutValue','OutTransFee','InCnt','InAmount','InValue','InTransFee']]
dfout

In [ ]:
dfout.fillna(0, inplace=True)

In [ ]:
dfout

In [ ]:
# dfout = pd.read_csv(prefix+'/raw_data/nodes.csv')

In [ ]:
with open(prefix+'/raw_data/suspicious_label/nftgraph_suspicious_labels.txt','r') as f:
    line = f.read()
    labels = eval(line)

In [ ]:
def getlabel(s):
    return 1 if s in labels else 0

In [ ]:
dfout['label'] = dfout['addr'].apply(getlabel)

In [ ]:
dfout

In [ ]:
dfout['label'].sum()

In [ ]:
dfout.to_csv('./nodes.csv',index=False)

In [ ]:
dfout[dfout['label']==1].head(4600)